<a href="https://colab.research.google.com/github/seilsepour/SSTSA/blob/main/OverAll_Deep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


How to calculate metrics
https://datascience.stackexchange.com/questions/45165/how-to-get-accuracy-f1-precision-and-recall-for-a-keras-model


https://stackabuse.com/python-for-nlp-movie-sentiment-analysis-using-deep-learning-in-keras/



Increasing the number of layers

ٌWord2vec Dimension = 500

In [ ]:
%tensorflow_version 1.x

In [ ]:
import tensorflow
print(tensorflow.__version__)

In [ ]:

topicNumber = 70

In [ ]:
#pip install keras --user

# Configurations

In [ ]:
import numpy as np
import pandas as pd

import string
from __future__ import print_function

import keras
from keras.callbacks import EarlyStopping
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense, Dropout, Embedding, LSTM, GRU, Bidirectional, Activation, Conv1D, MaxPooling1D, GlobalMaxPooling1D, BatchNormalization
#from keras.optimizers import SGD
from keras.layers import SpatialDropout1D, Flatten
from keras.callbacks import Callback
from keras.callbacks import ReduceLROnPlateau
from keras import backend
from keras.initializers import Constant
from keras.regularizers import l2
from sklearn.metrics import precision_recall_fscore_support as score

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from matplotlib import pyplot as plt
from matplotlib.pyplot import plot



import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


In [ ]:
allTrainingData = '/MYDRIVE/My Drive/Colab Notebooks/SecondPaper/data/70-with-labels.xlsx'

In [ ]:
w2vMovieTxt = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/w2vMovie-MR-99-07-25.txt'
w2vMovieM = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/w2vMovie-MR-99-07-25.h5'

lstmModelTxt = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/lstm-99-07-25.txt'
lstmModelH = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/lstm-99-07-25.h5'

gruModelTxt = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/gru-99-07-25.txt'
gruModelH = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/gru-99-07-25.h5'

bilstmModelTxt = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/bilstm-99-07-25.txt'
bilstmModelH = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/bilstm-99-07-25.h5'

conv1ModelTxt = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/conv1-99-07-25.txt'
conv1ModelH = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/conv1-99-07-25.h5'

CLModelTxt = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/Cl-99-07-25.txt'
CLModelH = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/Cl-99-07-25.h5'

#This file includes tweets and topic number
tweetsTopics = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/Tweets_Topics_990725.csv'


mr8Similar = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/SimilarDocsByProposedMethod/Topics_8_MostSimilar_PM.csv'



pangLabelledDataset = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/MovieReview-Preprocessed-Pang-Ver2-Labelled.csv'

##Hyper parameters

In [ ]:
#Making Word2vec
embeddingDim = 500
maxLen = embeddingDim
testSize=0.2
windowSize=5
minCount=1
iterCount=10
sG = 0 #skip gram wil be used


#Making Model
validationSplit = 0.2
#maxFeatures = numWords
maxLen = 500
embeddingSize = 500
isTrainable = False
batchSize = 16



epochsNum = 20

unitsNum = 20


# Convolution
kernelSize = 4

filtersNum = 32


poolSize = 2
stridesNum = 1
hidden_dims = 100

# LSTM
lstm_output_size = 70

#Login to Google Drive

In [ ]:
#No need in local
from google.colab import drive
drive.mount('/MYDRIVE', force_remount=True)

# Sentiment Analysis by W2V Model

## Loading all Training labelled data

In [ ]:
allTrainingData

In [ ]:
dfALLTraining = pd.read_excel(allTrainingData)
print(dfALLTraining.columns)

In [ ]:
dfALLTraining = dfALLTraining[['id','PredictedLabel','ActualText']]
dfALLTraining.columns = ["docid","Sentiment","Text"]


In [ ]:
print(dfALLTraining.head(3))


In [ ]:
print(len(dfALLTraining[dfALLTraining["Sentiment"] == 0]))
print(len(dfALLTraining[dfALLTraining['Sentiment'] == 1]))

## Preprocessing

In [ ]:
dfALLTraining.head(3)

In [ ]:
from string import punctuation
from os import listdir
from nltk.corpus import stopwords
from pickle import dump

def clean_doc(doc):
	# split into tokens by white space
	tokens = doc.split()
	# remove punctuation from each token
	table = str.maketrans('', '', punctuation)
	tokens = [w.translate(table) for w in tokens]
	# remove remaining tokens that are not alphabetic
	tokens = [word for word in tokens if word.isalpha()]
	# filter out stop words
	stop_words = set(stopwords.words('english'))
	tokens = [w for w in tokens if not w in stop_words]
	# filter out short tokens
	tokens = [word for word in tokens if len(word) > 1]
	tokens = ' '.join(tokens)
	return tokens

In [ ]:
for index, row in dfALLTraining.iterrows():
     dfALLTraining.at[index,'Text']= clean_doc(row['Text'])


In [ ]:
dfALLTraining.head(3)

In [ ]:
print(dfALLTraining.nunique())

In [ ]:
print(max(dfALLTraining['docid']))
#print(max(dfALLTraining['index']))

In [ ]:
dfTrainingMovies = dfALLTraining

In [ ]:
numNegs = len(dfTrainingMovies[dfTrainingMovies['Sentiment'] == 0])
print(numNegs)
numNegs = len(dfTrainingMovies[dfTrainingMovies['Sentiment'] == 1])
print(numNegs)

In [ ]:
dfTrainingMovies.columns

## Making Word2Vec Model of movies

In [ ]:
#embeddingDim = 100
#Remove repeated rows
print(dfTrainingMovies.head())
dfMoviesNoRep = dfTrainingMovies
#dfMoviesNoRep.columns = ['i','TweetId', 'Tweet', 'Sentiment']
print(dfMoviesNoRep.head(3))
#dfMoviesNoRep.sort_values(["TweetId"], inplace = True)
print(dfMoviesNoRep.head(3))
# dropping ALL duplicte values
#dfMoviesNoRep.drop_duplicates(subset ="TweetId",
 #                    keep = 'first', inplace = True)
print(len(dfMoviesNoRep))

In [ ]:
numNegs = len(dfMoviesNoRep[dfMoviesNoRep['Sentiment'] == 0])
print(numNegs)

In [ ]:
numPoss = len(dfMoviesNoRep[dfMoviesNoRep['Sentiment'] == 1])
print(numPoss)

In [ ]:
diff = numNegs - numPoss

In [ ]:
# Making a balance dataset with equal number of positive and negative tweets
df1 = dfMoviesNoRep[dfMoviesNoRep['Sentiment'] == 1].sample(diff)
#df2 = dfMoviesNoRep[dfMoviesNoRep['Sentiment'] == 0].sample(numPoss)
df = dfMoviesNoRep.append(df1)
print(len(df[df['Sentiment'] == 1]))
print(len(df[df['Sentiment'] == 0]))
print(df.head())
print(len(df))

Analyzing Probability

In [ ]:
from sklearn.model_selection import train_test_split
x_train_m, x_test_m, y_train_m, y_test_m = train_test_split( df['Text'], df['Sentiment'], test_size=0.2, random_state=10)


In [ ]:
y_train_m

In [ ]:
print(len(x_test_m))
print(len(x_train_m))

In [ ]:
print(len(y_test_m[y_test_m == 0]))
print(len(y_test_m[y_test_m == 1]))

In [ ]:
print(len(y_train_m[y_train_m == 0]))
print(len(y_train_m[y_train_m == 1]))

In [ ]:
reviewMovieLines = list()
for line in x_train_m:
  reviewMovieLines.append( str(line).split() )

for line in x_test_m:
  reviewMovieLines.append( str(line).split() )

print(len(reviewMovieLines))

In [ ]:
import gensim


w2vMovie = gensim.models.Word2Vec( sentences=reviewMovieLines, size=embeddingDim,  window=windowSize, workers=4, min_count=minCount, iter=iterCount, sg = sG )

w2vGensimModel = gensim.models.Word2Vec()
#vocab size

words = list( w2vMovie.wv.vocab )

print('Vocabulary size: %d' % len(words))

In [ ]:
w2vMovieTxt

In [ ]:
w2vMovie.wv.save_word2vec_format( w2vMovieTxt, binary = False)
w2vMovie.save(w2vMovieM)

In [ ]:
w2vMovie.wv.most_similar('film')

# Desining new network by w2v

In [ ]:
import os
embeddingsIndex = {}
f = open(os.path.join('', w2vMovieTxt ), encoding = "utf-8")

for line in f:
  values = line.split()
  word = values[0]
  coefs = np.asarray(values[1:])
  embeddingsIndex[word] = coefs
f.close()

In [ ]:
tokenizerObj = Tokenizer()
tokenizerObj.fit_on_texts(reviewMovieLines)
sequences = tokenizerObj.texts_to_sequences(reviewMovieLines)

print(type(sequences))

reviewPad = pad_sequences( sequences, maxlen=maxLen)

wordIndex = tokenizerObj.word_index
print('Found %s unique tokens.' % len(wordIndex))

#reviewPad = pad_sequences( sequences, maxlen=maxLen)
#sentiment = df ['Sentiment'].values
print(type(reviewPad))
print(len(reviewPad))

x_train_pad = reviewPad[ : len(x_train_m)]
x_test_pad = reviewPad[ len(x_train_m) : ]
y_train = y_train_m
y_test = y_test_m

print(type(x_train_pad))

print('Shape of x_train_pad tensor:' , len(x_train_pad))
print('Shape of y_train tensor:' , len(y_train))


print('Shape of x_test_pad tensor:' , len(x_test_pad))
print('Shape of y_test tensor:' , len( y_test ))


In [ ]:
x = y_test[y_test == 0]
print(len(x))

In [ ]:
x = y_test[y_test == 1]
print(len(x))

In [ ]:
x = y_train[y_train == 0]
print(len(x))
x = y_train[y_train == 1]
print(len(x))

In [ ]:
embeddingDim

In [ ]:
numWords = len(wordIndex) + 1
print(numWords)
embeddingMatrix = np.zeros( (numWords, embeddingDim) )
#print(embeddingMatrix.shape)
for word, i in wordIndex.items():
  if i > numWords:
    continue
  embeddingVector = embeddingsIndex.get(word)
  if embeddingVector is not None:
    # words not found in embedding index will be all-zeros.
    embeddingMatrix[i] = embeddingVector

In [ ]:
print(embeddingMatrix.shape)

In [ ]:
max_features = numWords


# CNN+GRU


In [ ]:
import tensorflow as tf
epochsNum = 20
batchSize = 16

filtersNum = 32

max_features = numWords

# Convolution

print('Build model...')

modelCGRU = Sequential(name='CNNGRU')

modelCGRU.add(Embedding(numWords, embeddingDim ,
                      embeddings_initializer = Constant( embeddingMatrix),
                      input_length=maxLen,
                      trainable = isTrainable))


modelCGRU.add(Conv1D(filters= filtersNum, kernel_size= 4, strides= stridesNum, padding='same', activation='relu'))
modelCGRU.add(Dropout(0.1))

modelCGRU.add(Conv1D(filters= filtersNum, kernel_size= 4, strides= stridesNum, padding='same', activation='relu'))
modelCGRU.add(Dropout(0.1))

modelCGRU.add(MaxPooling1D(pool_size = 2))

modelCGRU.add(Dropout(0.1))
modelCGRU.add(GRU(10, dropout=0.1 ))
modelCGRU.add(Dropout(0.1))

modelCGRU.add(Dense(5, activation='relu'))

modelCGRU.add(Dense(1,activation='sigmoid'))



modelCGRU.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
print('Train...')

historyCGRU = modelCGRU.fit(x_train_pad, y_train,
          batch_size=batchSize,
          epochs=epochsNum,
          validation_data=(x_test_pad, y_test), verbose = 2 )

In [ ]:
testLossCGRU, testAccCGRU = modelCGRU.evaluate(x_test_pad, y_test, batch_size=batchSize, verbose=1)
print('Test loss:', testLossCGRU)
print('Test accuracy:', testAccCGRU)

trainLossCGRU, trainAccCGRU = modelCGRU.evaluate(x_train_pad, y_train, batch_size=batchSize, verbose=1)
print('Train loss:', trainLossCGRU)
print('Train accuracy:', trainAccCGRU)

In [ ]:
plt.title('Accuracy')
plt.plot(historyCGRU.history['accuracy'], label='train')
plt.plot(historyCGRU.history['val_accuracy'], label='test')
plt.legend()
plt.show();

In [ ]:
plt.title('Loss')
plt.plot(historyCGRU.history['loss'], label='train')
plt.plot(historyCGRU.history['val_loss'], label='test')
plt.legend()
plt.show();

In [ ]:
import tensorflow
print(tensorflow.__version__)

In [ ]:
y_val_pred_cg=modelCGRU.predict_classes(x_test_pad)
#y_val_pred_cg = modelCGRU.predict(x_test_pad)
precisionCGRU, recallCGRU, fscoreCGRU, supportCGRU = score(y_test, y_val_pred_cg)

print('precision: {}'.format(precisionCGRU))
print('recall: {}'.format(recallCGRU))
print('fscore: {}'.format(fscoreCGRU))
print('support: {}'.format(supportCGRU))

In [ ]:
from sklearn.metrics import cohen_kappa_score

cohen_score = cohen_kappa_score(y_test, y_val_pred_cg)

print(cohen_score)

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
labels = [0,1]
def cm_analysis(y_true, y_pred, labels, ymap=None, figsize=(10,10)):
    """
    Generate matrix plot of confusion matrix with pretty annotations.
    The plot image is saved to disk.
    args:
      y_true:    true label of the data, with shape (nsamples,)
      y_pred:    prediction of the data, with shape (nsamples,)
      filename:  filename of figure file to save
      labels:    string array, name the order of class labels in the confusion matrix.
                 use `clf.classes_` if using scikit-learn models.
                 with shape (nclass,).
      ymap:      dict: any -> string, length == nclass.
                 if not None, map the labels & ys to more understandable strings.
                 Caution: original y_true, y_pred and labels must align.
      figsize:   the size of the figure plotted.
    """
    if ymap is not None:
        y_pred = [ymap[yi] for yi in y_pred]
        y_true = [ymap[yi] for yi in y_true]
        labels = [ymap[yi] for yi in labels]
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    cm_sum = np.sum(cm, axis=1, keepdims=True)
    cm_perc = cm / cm_sum.astype(float) * 100
    annot = np.empty_like(cm).astype(str)
    nrows, ncols = cm.shape
    for i in range(nrows):
        for j in range(ncols):
            c = cm[i, j]
            p = cm_perc[i, j]
            if i == j:
                s = cm_sum[i]
                annot[i, j] = '%.1f%%\n%d/%d' % (p, c, s)
            elif c == 0:
                annot[i, j] = ''
            else:
                annot[i, j] = '%.1f%%\n%d' % (p, c)
    cm = pd.DataFrame(cm, index=labels, columns=labels)
    cm.index.name = 'Actual'
    cm.columns.name = 'Predicted'
    #fig, ax = plt.subplots(figsize=figsize)
    fig, ax = plt.subplots()
    sns.heatmap(cm, annot=annot, fmt='', ax=ax)
    #plt.savefig(filename)
    plt.show()

cm_analysis(y_test, y_val_pred_cg, labels, ymap=None, figsize=(5,5))

# CNN+BIGRU
sgd

In [ ]:
from keras.layers import GlobalMaxPooling1D

max_features = numWords





# Convolution

print('Build model...')

modelCBIGRU = Sequential(name='CNNBIGRU')

modelCBIGRU.add(Embedding(numWords, embeddingDim ,
                      embeddings_initializer = Constant( embeddingMatrix),
                      input_length=maxLen,
                      trainable = isTrainable))

#modelCGRU.add(Dropout(0.5))

modelCBIGRU.add(Conv1D(filters= filtersNum, kernel_size= 4, strides= stridesNum, padding='same', activation='relu'))


modelCBIGRU.add(MaxPooling1D(pool_size = 2))
modelCBIGRU.add(Dropout(0.1))
modelCBIGRU.add(Bidirectional(GRU(10, dropout=0.1 )))
modelCBIGRU.add(BatchNormalization())
modelCBIGRU.add(Dropout(0.5))




#modelCGRU_s.add(Dropout(0.6))

'''
Good for big dataset
modelCGRU_s.add(Conv1D(filters= filtersNum, kernel_size= 3, strides= stridesNum, padding='same', activation='relu'))
modelCGRU_s.add(Dropout(0.6))
modelCGRU_s.add(MaxPooling1D(pool_size=2))
modelCGRU_s.add(LSTM(32))
modelCGRU_s.add(Dropout(0.6))
'''
modelCBIGRU.add(Dense(1,activation='sigmoid'))

modelCBIGRU.compile(loss='binary_crossentropy',
              optimizer= 'adam',
              metrics=['accuracy'])
print('Train...')
rlrp = ReduceLROnPlateau(monitor='val_loss',
                                  factor=0.2,
                                  patience=4,
                                  verbose=1,
                                  mode='auto',
                                  min_delta=0.0001,
                                  cooldown=8,
                                  min_lr=0)


In [ ]:
historyCBIGRU = modelCBIGRU.fit(x_train_pad, y_train,
          batch_size=batchSize,
          epochs=epochsNum,
          validation_data=(x_test_pad, y_test), verbose = 2 )#, callbacks=[rlrp])

In [ ]:
modelCBIGRU.summary()

In [ ]:
testLossCBIGRU, testAccCBIGRU = modelCBIGRU.evaluate(x_test_pad, y_test, batch_size=batchSize, verbose=1)
print('Test loss:', testLossCBIGRU)
print('Test accuracy:', testAccCBIGRU)

trainLossCBIGRU, trainAccCBIGRU = modelCBIGRU.evaluate(x_train_pad, y_train, batch_size=batchSize, verbose=1)
print('Train loss:', trainLossCBIGRU)
print('Train accuracy:', trainAccCBIGRU)

In [ ]:
modelCBIGRU.summary()

In [ ]:
plt.title('Accuracy')
plt.plot(historyCBIGRU.history['accuracy'], label='train')
plt.plot(historyCBIGRU.history['val_accuracy'], label='test')
plt.legend()
plt.show();

In [ ]:
plt.title('Loss')
plt.plot(historyCBIGRU.history['loss'], label='train')
plt.plot(historyCBIGRU.history['val_loss'], label='test')
plt.legend()
plt.show();

In [ ]:
y_val_pred_cbgru=modelCBIGRU.predict_classes(x_test_pad)
precisionCBIGRU, recallCBIGRU, fscoreCBIGRU, supportCBIGRU = score(y_test, y_val_pred_cbgru)

print('precision: {}'.format(precisionCBIGRU))
print('recall: {}'.format(recallCBIGRU))
print('fscore: {}'.format(fscoreCBIGRU))
print('support: {}'.format(supportCBIGRU))


In [ ]:
from sklearn.metrics import cohen_kappa_score
cohen_score = cohen_kappa_score(y_test, y_val_pred_cbgru)
print(cohen_score)

In [ ]:
len(y_test[y_test==1])

# CNN+BiLSTM

In [ ]:
# Embedding

max_features = numWords


# Convolution




print('Build model...')

modelCBiLSTM = Sequential(name='CNNBiLSTM')
modelCBiLSTM.add(Embedding(numWords, embeddingDim ,
                      embeddings_initializer = Constant( embeddingMatrix),
                      input_length=maxLen,
                      trainable = isTrainable))


modelCBiLSTM.add(Conv1D(filters= filtersNum, kernel_size= 4, strides= stridesNum, padding='same', activation='relu'))


modelCBiLSTM.add(Dropout(0.1))
modelCBiLSTM.add(MaxPooling1D(pool_size = 2))

modelCBiLSTM.add(Dropout(0.1))
modelCBiLSTM.add(Bidirectional(LSTM(10, dropout=0.1 )))
modelCBiLSTM.add(BatchNormalization())
modelCBiLSTM.add(Dropout(0.1))

modelCBiLSTM.add(Dense(1,activation='sigmoid'))






modelCBiLSTM.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


#modelCBiLSTM.summary()

print('Train...')

rlrp = ReduceLROnPlateau(monitor='val_loss',
                                  factor=0.1,
                                  patience=8,
                                  verbose=1,
                                  mode='auto',
                                  min_delta=0.0001,
                                  cooldown=8,
                                  min_lr=0)
#lrm = LearningRateMonitor()

historyCBiLSTM = modelCBiLSTM.fit(x_train_pad, y_train,
          batch_size=batchSize,
          epochs=epochsNum,
          validation_data=(x_test_pad, y_test), verbose = 2 )#, callbacks=[rlrp])



In [ ]:
testLossCBiLSTM, testAccCBiLSTM = modelCBiLSTM.evaluate(x_test_pad, y_test, batch_size=batchSize, verbose=1)
print('Test loss:', testLossCBiLSTM)
print('Test accuracy:', testAccCBiLSTM)

In [ ]:
trainLossCBiLSTM, trainAccCBiLSTM = modelCBiLSTM.evaluate(x_train_pad, y_train, batch_size=batchSize, verbose=1)
print('Train loss:', trainLossCBiLSTM)
print('Train accuracy:', trainAccCBiLSTM)

In [ ]:
modelCBiLSTM.summary()

In [ ]:
plt.title('Accuracy')
plt.plot(historyCBiLSTM.history['accuracy'], label='train')
plt.plot(historyCBiLSTM.history['val_accuracy'], label='test')
plt.legend()
plt.show();

In [ ]:
plt.title('Loss')
plt.plot(historyCBiLSTM.history['loss'], label='train')
plt.plot(historyCBiLSTM.history['val_loss'], label='test')
plt.legend()
plt.show();

In [ ]:
y_val_pred_cbl = modelCBiLSTM.predict_classes(x_test_pad)
precisionCBiLSTM, recallCBiLSTM, fscoreCBiLSTM, supportCBiLSTM = score(y_test, y_val_pred_cbl)

print('precision: {}'.format(precisionCBiLSTM))
print('recall: {}'.format(recallCBiLSTM))
print('fscore: {}'.format(fscoreCBiLSTM))
print('support: {}'.format(supportCBiLSTM))




In [ ]:
from sklearn.metrics import cohen_kappa_score

cohen_score = cohen_kappa_score(y_test, y_val_pred_cbl)

print(cohen_score)

# CNN+LSTM

In [ ]:
# Embedding



max_features = numWords





# Convolution



'''
Note:
batch_size is highly sensitive.
Only 2 epochs are needed as the dataset is very small.
'''
print('Build model...')

modelCL = Sequential(name='CNNLSTM')
modelCL.add(Embedding(numWords, embeddingDim ,
                      embeddings_initializer = Constant( embeddingMatrix),
                      input_length=maxLen,
                      trainable = isTrainable))


modelCL.add(Conv1D(filters= filtersNum, kernel_size= 4, strides= stridesNum, padding='same', activation='relu'))


modelCL.add(Dropout(0.1))
modelCL.add(MaxPooling1D(pool_size = 2))

modelCL.add(Dropout(0.1))
modelCL.add(LSTM(10, dropout=0.1 ))
modelCL.add(BatchNormalization())
modelCL.add(Dropout(0.1))



modelCL.add(Dense(1,activation='sigmoid'))





modelCL.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


#modelCL.summary()

print('Train...')

rlrp = ReduceLROnPlateau(monitor='val_loss',
                                  factor=0.1,
                                  patience=8,
                                  verbose=1,
                                  mode='auto',
                                  min_delta=0.0001,
                                  cooldown=8,
                                  min_lr=0)
#lrm = LearningRateMonitor()

historyCL = modelCL.fit(x_train_pad, y_train,
          batch_size=batchSize,
          epochs=epochsNum,
          validation_data=(x_test_pad, y_test), verbose = 2 )#, callbacks=[rlrp])


In [ ]:
testLossCL, testAccCL = modelCL.evaluate(x_test_pad, y_test, batch_size=batchSize, verbose=1)
print('Test loss:', testLossCL)
print('Test accuracy:', testAccCL)

In [ ]:
trainLossCL, trainAccCL = modelCL.evaluate(x_train_pad, y_train, batch_size=batchSize, verbose=1)
print('Train loss:', trainLossCL)
print('Train accuracy:', trainAccCL)

In [ ]:
#modelCL.save(CLModelTxt)
#modelCL.save(CLModelH)


In [ ]:
modelCL.summary()

In [ ]:
plt.title('Accuracy')
plt.plot(historyCL.history['accuracy'], label='train')
plt.plot(historyCL.history['val_accuracy'], label='test')
plt.legend()
plt.show();

In [ ]:
plt.title('Loss')
plt.plot(historyCL.history['loss'], label='train')
plt.plot(historyCL.history['val_loss'], label='test')
plt.legend()
plt.show();

In [ ]:
y_val_pred_cl=modelCL.predict_classes(x_test_pad)
precisionCL, recallCL, fscoreCL, supportCL = score(y_test, y_val_pred_cl)

print('precision: {}'.format(precisionCL))
print('recall: {}'.format(recallCL))
print('fscore: {}'.format(fscoreCL))
print('support: {}'.format(supportCL))

In [ ]:
from sklearn.metrics import cohen_kappa_score

cohen_score = cohen_kappa_score(y_test, y_val_pred_cl)

print(cohen_score)

# LSTM

In [ ]:
max_features = numWords

# LSTM

batchSize = 16




#
print('Build model...')
modelLSTM = Sequential()






embeddingLayer = Embedding( numWords,
                           embeddingDim,
                           embeddings_initializer = Constant( embeddingMatrix),
                           input_length = maxLen,
                           trainable = isTrainable)
modelLSTM.add(embeddingLayer)
modelLSTM.add(Dropout(0.1))
modelLSTM.add(LSTM(10, dropout=0.1))

modelLSTM.add(Dropout(0.1))

modelLSTM.add(Dense(1,activation='sigmoid'))


modelLSTM.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])



#modelLSTM.summary()

rlrp = ReduceLROnPlateau(monitor='val_loss',
                                  factor=0.1,
                                  patience=8,
                                  verbose=1,
                                  mode='auto',
                                  min_delta=0.0001,
                                  cooldown=8,
                                  min_lr=0)

historyLSTM = modelLSTM.fit(x_train_pad, y_train,
          batch_size=batchSize,
          epochs=epochsNum,
          validation_data=(x_test_pad, y_test),  verbose = 2)#, callbacks=[rlrp])



In [ ]:
testLossLSTM, testAccLSTM = modelLSTM.evaluate(x_test_pad, y_test,
                            batch_size=batchSize)
print('Test loss:', testLossLSTM)
print('Test accuracy:', testAccLSTM)

In [ ]:
trainLossLSTM, trainAccLSTM = modelLSTM.evaluate(x_train_pad, y_train, batch_size=batchSize, verbose=1)
print('Train loss:', trainLossLSTM)
print('Train accuracy:', trainAccLSTM)

In [ ]:
#modelLSTM.save(lstmModelTxt)
#modelLSTM.save(lstmModelH)

In [ ]:
modelLSTM.summary()

In [ ]:
plt.title('Accuracy')
plt.plot(historyLSTM.history['accuracy'], label='train')
plt.plot(historyLSTM.history['val_accuracy'], label='test')
plt.legend()
plt.show();

In [ ]:
plt.title('Loss')
plt.plot(historyLSTM.history['loss'], label='train')
plt.plot(historyLSTM.history['val_loss'], label='test')
plt.legend()
plt.show();

In [ ]:
y_val_pred_l=modelLSTM.predict_classes(x_test_pad)
precisionLSTM, recallLSTM, fscoreLSTM, supportLSTM = score(y_test, y_val_pred_l)

print('precision: {}'.format(precisionLSTM))
print('recall: {}'.format(recallLSTM))
print('fscore: {}'.format(fscoreLSTM))
print('support: {}'.format(supportLSTM))

In [ ]:
from sklearn.metrics import cohen_kappa_score

cohen_score = cohen_kappa_score(y_test, y_val_pred_l)

print(cohen_score)

# GRU

In [ ]:
print('Build model...')


modelGRU = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
embeddingLayer = Embedding( numWords,
                           embeddingDim,
                           embeddings_initializer = Constant( embeddingMatrix),
                           input_length = maxLen,
                           trainable = isTrainable)

modelGRU.add(embeddingLayer)

modelGRU.add(Dropout(0.1))
modelGRU.add(GRU(units= 10 , dropout = 0.1))

modelGRU.add(Dropout(0.1))
modelGRU.add(Dense(1,activation='sigmoid'))




modelGRU.compile(loss='binary_crossentropy',
              optimizer= 'adam',
              metrics=['accuracy'])

modelGRU.summary()


rlrp = ReduceLROnPlateau(monitor='val_loss',
                                  factor=0.1,
                                  patience=8,
                                  verbose=1,
                                  mode='auto',
                                  min_delta=0.0001,
                                  cooldown=8,
                                  min_lr=0)

historyGRU = modelGRU.fit(x_train_pad, y_train,
          batch_size=batchSize,
          epochs= epochsNum,
          validation_data=(x_test_pad, y_test),
          verbose = 2)  # , callbacks=[rlrp])



In [ ]:
testLossGRU, testAccGRU = modelGRU.evaluate(x_test_pad, y_test,
                            batch_size=batchSize)
print('Test loss:', testLossGRU)
print('Test accuracy:', testAccGRU)

In [ ]:
trainLossGRU, trainAccGRU = modelGRU.evaluate(x_train_pad, y_train,
                            batch_size=batchSize)
print('Train loss:', trainLossGRU)
print('Train accuracy:', trainAccGRU)

In [ ]:
#modelGRU.save(gruModelTxt)
#modelGRU.save(gruModelH)

In [ ]:
modelGRU.summary()

In [ ]:
plt.title('Accuracy')
plt.plot(historyGRU.history['accuracy'], label='train')
plt.plot(historyGRU.history['val_accuracy'], label='test')
plt.legend()
plt.show();

In [ ]:
plt.title('Loss')
plt.plot(historyGRU.history['loss'], label='train')
plt.plot(historyGRU.history['val_loss'], label='test')
plt.legend()
plt.show();

In [ ]:
from sklearn.metrics import precision_recall_fscore_support as score


y_val_pred_g=modelGRU.predict_classes(x_test_pad)
precisionGRU, recallGRU, fscoreGRU, supportGRU = score(y_test, y_val_pred_g)

print('precision: {}'.format(precisionGRU))
print('recall: {}'.format(recallGRU))
print('fscore: {}'.format(fscoreGRU))
print('support: {}'.format(supportGRU))

In [ ]:
from sklearn.metrics import cohen_kappa_score

cohen_score = cohen_kappa_score(y_test, y_val_pred_g)

print(cohen_score)

# BIGRU

In [ ]:
print('Build model...')


modelBIGRU = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
embeddingLayer = Embedding( numWords,
                           embeddingDim,
                           embeddings_initializer = Constant( embeddingMatrix),
                           input_length = maxLen,
                           trainable = isTrainable)

modelBIGRU.add(embeddingLayer)
modelBIGRU.add(Dropout(0.1))
modelBIGRU.add(Bidirectional(GRU(units=10, dropout = 0.1)))
modelBIGRU.add(BatchNormalization())

modelBIGRU.add(Dropout(0.1))
modelBIGRU.add(Dense(1,activation='sigmoid'))



modelBIGRU.compile(loss='binary_crossentropy',
              optimizer= 'adam',
              metrics=['accuracy'])


historyBIGRU = modelBIGRU.fit(x_train_pad, y_train,
          batch_size=batchSize,
          epochs=epochsNum,
          validation_data=(x_test_pad, y_test),
          verbose = 2)  # , callbacks=[rlrp])



In [ ]:
testLossBIGRU, testAccBIGRU = modelBIGRU.evaluate(x_test_pad, y_test,
                            batch_size=batchSize)
print('Test loss:', testLossBIGRU)
print('Test accuracy:', testAccBIGRU)

In [ ]:
trainLossBIGRU, trainAccBIGRU = modelBIGRU.evaluate(x_train_pad, y_train,
                            batch_size=batchSize)
print('Train loss:', trainLossBIGRU)
print('Train accuracy:', trainAccBIGRU)

In [ ]:
modelBIGRU.summary()

In [ ]:
plt.title('Accuracy')
plt.plot(historyBIGRU.history['accuracy'], label='train')
plt.plot(historyBIGRU.history['val_accuracy'], label='test')
plt.legend()
plt.show();

In [ ]:
plt.title('Loss')
plt.plot(historyBIGRU.history['loss'], label='train')
plt.plot(historyBIGRU.history['val_loss'], label='test')
plt.legend()
plt.show();

In [ ]:
from sklearn.metrics import precision_recall_fscore_support as score


y_val_pred_bg=modelBIGRU.predict_classes(x_test_pad)
precisionbGRU, recallbGRU, fscorebGRU, supportbGRU = score(y_test, y_val_pred_bg)

print('precision: {}'.format(precisionbGRU))
print('recall: {}'.format(recallbGRU))
print('fscore: {}'.format(fscorebGRU))
print('support: {}'.format(supportbGRU))

In [ ]:
from sklearn.metrics import cohen_kappa_score

cohen_score = cohen_kappa_score(y_test, y_val_pred_bg)

print(cohen_score)

# BILSTM

In [ ]:
maxFeatures = numWords

unitsNumber = 20







#model.compile(..., optimizer=opt)
print('Build model...')
modelBiLSTM = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
embeddingLayer = Embedding( numWords,
                           embeddingDim,
                           embeddings_initializer = Constant( embeddingMatrix),
                           input_length = maxLen,
                           trainable = isTrainable)

modelBiLSTM.add(embeddingLayer)

modelBiLSTM.add(Dropout(0.1))

modelBiLSTM.add(Bidirectional(LSTM(10, dropout = 0.1)))

modelBiLSTM.add(Dropout(0.1))

modelBiLSTM.add(Dense(1,activation='sigmoid'))

modelBiLSTM.compile(loss='binary_crossentropy',optimizer= 'adam', metrics=['accuracy'])


#modelBiLSTM.summary()

rlrp = ReduceLROnPlateau(monitor='val_loss',
                                  factor=0.1,
                                  patience=8,
                                  verbose=1,
                                  mode='auto',
                                  min_delta=0.0001,
                                  cooldown=8,
                                  min_lr=0)

historyBILSTM = modelBiLSTM.fit(x_train_pad, y_train,
          batch_size=batchSize,
          epochs=epochsNum,
          validation_data=(x_test_pad, y_test),
          verbose = 2 )#, callbacks=[rlrp])



In [ ]:
testLossBilstm, testAccBilstm = modelBiLSTM.evaluate(x_test_pad, y_test,
                            batch_size=batchSize)
print('Test score:', testLossBilstm)
print('Test accuracy:', testAccBilstm)

In [ ]:
trainLossBilstm, trainAccBilstm = modelBiLSTM.evaluate(x_train_pad, y_train,
                            batch_size=batchSize)
print('Train score:', trainLossBilstm)
print('Train accuracy:', trainAccBilstm)

In [ ]:
#modelBiLSTM.save(bilstmModelTxt)
#modelBiLSTM.save(bilstmModelH)

In [ ]:
modelBiLSTM.summary()

In [ ]:
plt.title('Accuracy')
plt.plot(historyBILSTM.history['accuracy'], label='train')
plt.plot(historyBILSTM.history['val_accuracy'], label='test')
plt.legend()
plt.show();

In [ ]:
plt.title('Loss')
plt.plot(historyBILSTM.history['loss'], label='train')
plt.plot(historyBILSTM.history['val_loss'], label='test')
plt.legend()
plt.show();

In [ ]:
y_val_pred_bl=modelBiLSTM.predict_classes(x_test_pad)
precisionBilstm, recallBilstm, fscoreBilstm, supportBilstm = score(y_test, y_val_pred_bl)

print('precision: {}'.format(precisionBilstm))
print('recall: {}'.format(recallBilstm))
print('fscore: {}'.format(fscoreBilstm))
print('support: {}'.format(supportBilstm))

In [ ]:
from sklearn.metrics import cohen_kappa_score

cohen_score = cohen_kappa_score(y_test, y_val_pred_bl)

print(cohen_score)

# CNN

In [ ]:


# set parameters:
max_features = numWords
#maxlen = 400


print('Build model...')
modelConv1 = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
modelConv1.add(Embedding(numWords, embeddingDim ,
                      embeddings_initializer = Constant( embeddingMatrix),
                      input_length=maxLen,
                      trainable = isTrainable))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:

modelConv1.add(Conv1D(filters=filtersNum,
                 kernel_size=kernelSize,
                 padding='same',
                 activation='relu',
                 strides=1))
modelConv1.add(Dropout(0.2))

modelConv1.add(MaxPooling1D(pool_size=2))
modelConv1.add(Dropout(0.1))

modelConv1.add(Flatten())
modelConv1.add(Dense(10, activation='relu'))
modelConv1.add(Dropout(0.1))

modelConv1.add(BatchNormalization())

modelConv1.add(Dense(1, activation='sigmoid'))











modelConv1.compile(loss='binary_crossentropy',
              optimizer= 'adam',
              metrics=['accuracy'])

#modelConv1.summary()


historyConv1=modelConv1.fit(x_train_pad, y_train,
          batch_size=batchSize,
          epochs=epochsNum,
          validation_data=(x_test_pad, y_test), verbose = 2)


In [ ]:
testLossConv, testAccConv = modelConv1.evaluate(x_test_pad, y_test, batch_size=batchSize)
print('Test loss:', testLossConv)
print('Test accuracy:', testAccConv)

In [ ]:
trainLossConv, trainAccConv = modelConv1.evaluate(x_train_pad, y_train, batch_size=batchSize)
print('Train loss:', trainLossConv)
print('Train accuracy:', trainAccConv)

In [ ]:
#modelConv1.save(conv1ModelTxt)
#modelConv1.save(conv1mModelH)

In [ ]:
modelConv1.summary()

In [ ]:
plt.title('Accuracy')
plt.plot(historyConv1.history['accuracy'], label='train')
plt.plot(historyConv1.history['val_accuracy'], label='test')
plt.legend()
plt.show();

In [ ]:
plt.title('Loss')
plt.plot(historyConv1.history['loss'], label='train')
plt.plot(historyConv1.history['val_loss'], label='test')
plt.legend()
plt.show();

In [ ]:
y_val_pred_c =modelConv1.predict_classes(x_test_pad)
precisionConv1, recallConv1, fscoreConv1, supportConv1 = score(y_test, y_val_pred_c)

print('precision: {}'.format(precisionConv1))
print('recall: {}'.format(recallConv1))
print('fscore: {}'.format(fscoreConv1))
print('support: {}'.format(supportConv1))

In [ ]:
from sklearn.metrics import cohen_kappa_score

cohen_score = cohen_kappa_score(y_test, y_val_pred_c)

print(cohen_score)

# Evaluation

## Accuracy

In [ ]:
dicTestAccuracy={
      "LSTM"      :("{:.4f}".format(testAccLSTM)),
      "Bilstm":("{:.4f}".format(testAccBilstm)),
      "GRU"      :("{:.4f}".format(testAccGRU)),
      "CNN"     :("{:.4f}".format(testAccConv)),
      "CNN+LSTM"      :("{:.4f}".format(testAccCL)),
      "CNN+GRU"      :("{:.4f}".format(testAccCGRU)),
      "CNN+BiLSTM"      :("{:.4f}".format(testAccCBiLSTM))
      }

In [ ]:
dfTestAccuracy = pd.DataFrame.from_dict(dicTestAccuracy, orient='index', columns=["Accuracy"] )
dfTestAccuracy

In [ ]:
dfTestAccuracy["Accuracy"] = dfTestAccuracy["Accuracy"].astype(float)
dfTestAccuracy["Accuracy"].plot(kind="bar", grid=True , color=tuple(["g", "b", "r", "y", "k"]), width=0.3)
plt.title("Accuracy")

## Precision

In [ ]:
precisionLSTM

In [ ]:
dicPrecision={
      "LSTM"      :("{:.4f}".format(precisionLSTM[0])),
      "Bilstm":("{:.4f}".format(precisionBilstm[0])),
      "GRU"      :("{:.4f}".format(precisionGRU[0])),
      "CNN"     :("{:.4f}".format(precisionConv1[0])),
      "CNN+LSTM"      :("{:.4f}".format(precisionCL[0])),
      "CNN+GRU"      :("{:.4f}".format(precisionCGRU[0])),
      "CNN+BiLSTM"      :("{:.4f}".format(precisionCBiLSTM[0]))
      }


In [ ]:
dfPrecision = pd.DataFrame.from_dict(dicPrecision, orient='index', columns=["Precision"] )
dfPrecision

In [ ]:
dfPrecision["Precision"] = dfPrecision["Precision"].astype(float)
dfPrecision["Precision"].plot(kind="bar", grid=True , color=tuple(["g", "b", "r", "y", "k"]), width=0.3)
plt.title("Precision")

## Recall

In [ ]:
dicRecall={
      "LSTM"      :("{:.4f}".format(recallLSTM[0])),
      "Bilstm":("{:.4f}".format(recallBilstm[0])),
      "GRU"      :("{:.4f}".format(recallGRU[0])),
      "CNN"     :("{:.4f}".format(recallConv1[0])),
      "CNN+LSTM"      :("{:.4f}".format(recallCL[0])),
      "CNN+GRU"      :("{:.4f}".format(recallCGRU[0])),
      "CNN+BiLSTM"      :("{:.4f}".format(recallCBiLSTM[0]))
      }

In [ ]:
dfRecall = pd.DataFrame.from_dict(dicRecall, orient='index', columns=["Recall"] )
dfRecall

In [ ]:
dfRecall["Recall"] = dfRecall["Recall"].astype(float)
dfRecall["Recall"].plot(kind="bar", grid=True , color=tuple(["g", "b", "r", "y", "k"]), width=0.3)
plt.title("Recall")

## Fscore

In [ ]:
dicFscore={
      "LSTM"      :("{:.4f}".format(fscoreLSTM[0])),
      "Bilstm":("{:.4f}".format(fscoreBilstm[0])),
      "GRU"      :("{:.4f}".format(fscoreGRU[0])),
      "CNN"     :("{:.4f}".format(fscoreConv1[0])),
      "CNN+LSTM"      :("{:.4f}".format(fscoreCL[0])),
      "CNN+GRU"      :("{:.4f}".format(fscoreCGRU[0])),
      "CNN+BiLSTM"      :("{:.4f}".format(fscoreCBiLSTM[0]))
      }

In [ ]:
dfFscore = pd.DataFrame.from_dict(dicFscore, orient='index', columns=["Fscore"] )
dfFscore

In [ ]:
dfFscore["Fscore"] = dfFscore["Fscore"].astype(float)
dfFscore["Fscore"].plot(kind="bar", grid=True , color=tuple(["g", "b", "r", "y", "k"]), width=0.3)
plt.title("Fscore")

##ALL

positives

In [ ]:
dicALL={
      "Accuracy":[
                  ("{:.4f}".format(testAccLSTM)),
                  ("{:.4f}".format(testAccBilstm)),
                  ("{:.4f}".format(testAccGRU)),
                  ("{:.4f}".format(testAccBIGRU)),
                  ("{:.4f}".format(testAccConv)),
                  ("{:.4f}".format(testAccCL)),
                  ("{:.4f}".format(testAccCGRU)),
                  ("{:.4f}".format(testAccCBiLSTM)),
                  ("{:.4f}".format(testAccCBIGRU))
                 ],

      "Precision":[
                  ("{:.4f}".format(precisionLSTM[0])),
                  ("{:.4f}".format(precisionBilstm[0])),
                  ("{:.4f}".format(precisionGRU[0])),
                  ("{:.4f}".format(precisionbGRU[0])),
                  ("{:.4f}".format(precisionConv1[0])),
                  ("{:.4f}".format(precisionCL[0])),
                  ("{:.4f}".format(precisionCGRU[0])),
                  ("{:.4f}".format(precisionCBiLSTM[0])),
                  ("{:.4f}".format(precisionCBIGRU[0]))
                 ],

      "Recall":[
                ("{:.4f}".format(recallLSTM[0])),
                ("{:.4f}".format(recallBilstm[0])),
                ("{:.4f}".format(recallGRU[0])),
                ("{:.4f}".format(recallbGRU[0])),
                ("{:.4f}".format(recallConv1[0])),
                ("{:.4f}".format(recallCL[0])),
                ("{:.4f}".format(recallCGRU[0])),
                ("{:.4f}".format(recallCBiLSTM[0])),
                ("{:.4f}".format(recallCBIGRU[0]))
                 ],

      "F1":[
            ("{:.4f}".format(fscoreLSTM[0])),
            ("{:.4f}".format(fscoreBilstm[0])),
            ("{:.4f}".format(fscoreGRU[0])),
            ("{:.4f}".format(fscorebGRU[0])),
            ("{:.4f}".format(fscoreConv1[0])),
            ("{:.4f}".format(fscoreCL[0])),
            ("{:.4f}".format(fscoreCGRU[0])),
            ("{:.4f}".format(fscoreCBiLSTM[0])),
            ("{:.4f}".format(fscoreCBIGRU[0]))
           ],
     "Loss":[
            ("{:.4f}".format(testLossLSTM)),
            ("{:.4f}".format(testLossBilstm)),
            ("{:.4f}".format(testLossGRU)),
            ("{:.4f}".format(testLossBIGRU)),
            ("{:.4f}".format(testLossConv)),
            ("{:.4f}".format(testLossCL)),
            ("{:.4f}".format(testLossCGRU)),
            ("{:.4f}".format(testLossCBiLSTM)),
            ("{:.4f}".format(testLossCBIGRU))
           ]


        }


dfAllMeasures=pd.DataFrame.from_dict( dicALL , orient='index')#,index=['SIMPLE RNN','LSTM MODEL','GRU MODEL','SVR MODEL','SARIMAX MODEL'],
                                 #columns=["MSE","MAE"] )
dfAllMeasures.columns=['LSTM','Bilstm','GRU', 'BIGRU','Conv1','CNN+LSTM', 'CNN+GRU', 'CNN+BiLSTM', 'CNN+BIGRU']
dfAllMeasures = dfAllMeasures.T
dfAllMeasures

negatives

In [ ]:
dicALL={
      "Accuracy":[
                  ("{:.4f}".format(testAccLSTM)),
                  ("{:.4f}".format(testAccBilstm)),
                  ("{:.4f}".format(testAccGRU)),
                  ("{:.4f}".format(testAccBIGRU)),
                  ("{:.4f}".format(testAccConv)),
                  ("{:.4f}".format(testAccCL)),
                  ("{:.4f}".format(testAccCGRU)),
                  ("{:.4f}".format(testAccCBiLSTM)),
                  ("{:.4f}".format(testAccCBIGRU))
                 ],

      "Precision":[
                  ("{:.4f}".format(precisionLSTM[1])),
                  ("{:.4f}".format(precisionBilstm[1])),
                  ("{:.4f}".format(precisionGRU[1])),
                  ("{:.4f}".format(precisionbGRU[1])),
                  ("{:.4f}".format(precisionConv1[1])),
                  ("{:.4f}".format(precisionCL[1])),
                  ("{:.4f}".format(precisionCGRU[1])),
                  ("{:.4f}".format(precisionCBiLSTM[1])),
                  ("{:.4f}".format(precisionCBIGRU[1]))
                 ],

      "Recall":[
                ("{:.4f}".format(recallLSTM[1])),
                ("{:.4f}".format(recallBilstm[1])),
                ("{:.4f}".format(recallGRU[1])),
                ("{:.4f}".format(recallbGRU[1])),
                ("{:.4f}".format(recallConv1[1])),
                ("{:.4f}".format(recallCL[1])),
                ("{:.4f}".format(recallCGRU[1])),
                ("{:.4f}".format(recallCBiLSTM[1])),
                ("{:.4f}".format(recallCBIGRU[1]))
                 ],

      "F1":[
            ("{:.4f}".format(fscoreLSTM[1])),
            ("{:.4f}".format(fscoreBilstm[1])),
            ("{:.4f}".format(fscoreGRU[1])),
            ("{:.4f}".format(fscorebGRU[1])),
            ("{:.4f}".format(fscoreConv1[1])),
            ("{:.4f}".format(fscoreCL[1])),
            ("{:.4f}".format(fscoreCGRU[1])),
            ("{:.4f}".format(fscoreCBiLSTM[1])),
            ("{:.4f}".format(fscoreCBIGRU[1]))
           ],
     "Loss":[
            ("{:.4f}".format(testLossLSTM)),
            ("{:.4f}".format(testLossBilstm)),
            ("{:.4f}".format(testLossGRU)),
            ("{:.4f}".format(testLossBIGRU)),
            ("{:.4f}".format(testLossConv)),
            ("{:.4f}".format(testLossCL)),
            ("{:.4f}".format(testLossCGRU)),
            ("{:.4f}".format(testLossCBiLSTM)),
            ("{:.4f}".format(testLossCBIGRU))
           ]


        }


dfAllMeasures=pd.DataFrame.from_dict( dicALL , orient='index')#,index=['SIMPLE RNN','LSTM MODEL','GRU MODEL','SVR MODEL','SARIMAX MODEL'],
                                 #columns=["MSE","MAE"] )
dfAllMeasures.columns=['LSTM','Bilstm','GRU', 'BIGRU','Conv1','CNN+LSTM', 'CNN+GRU', 'CNN+BiLSTM', 'CNN+BIGRU']
dfAllMeasures = dfAllMeasures.T
dfAllMeasures